## Deep Fake Detection Challenge

### Data Processing

##### Imports

In [ ]:
import os
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np
import cv2
from typing import Tuple

##### Data Description

In [ ]:
train_videoes = "data/train_sample_videos"
test_videos = "data/test_videos"

print(f"Train Videoes: {len(os.listdir(train_videoes))}\nTest Vidoes: {len(os.listdir(test_videos))}")

In [ ]:
meta_data = pd.read_json(train_videoes+"/metadata.json").T
meta_data.head()

In [ ]:
def visualize_real_vs_fake(class_count: list) -> None:
    plt.pie(class_count, labels=["Real Videos", "Deep Fake Videos"], autopct='%.2f%%',
       wedgeprops={'linewidth': 2.5, 'edgecolor': 'white'},
       textprops={'size': 'large', 'fontweight': 'bold'})
    plt.title("Proportion of Real vs Deep Fake videos in the training dataset.", fontdict={'fontweight': 'bold'})
    plt.legend([f"Real Videos Count: {class_count[0]}", f"Deep Fake Videos Count: {class_count[1]}"], bbox_to_anchor=(0.5, 0.05), bbox_transform=plt.gcf().transFigure, loc="lower center", prop={'weight':'bold'})
    plt.savefig("images/pie_chart_class_proportions.jpg")
    plt.show()

In [ ]:
label_count_series = meta_data["label"].value_counts()
fake_count = label_count_series["FAKE"]
real_count = label_count_series["REAL"]

visualize_real_vs_fake([real_count, fake_count])

Quite skewed dataset. Might want to consider upsampling of real classes

In [ ]:
def get_frames_v1(paths: list, frames_each_video: int, video_amount: int) -> list:
    video_array = []
    frames_to_skip = 300/frames_each_video
    for idx, path in enumerate(paths): # bedre me enumerate her?
        if idx == video_amount:
            break
        vc = cv2.VideoCapture(path)
        video = []
        i = 0
        while vc.isOpened():
            i += 1
            ret, frame = vc.read()
            if ret and frame is not None:
                if i % frames_to_skip != 0:
                    continue
                if frame.shape[0] == 1920:
                    frame = frame.transpose(1, 0, 2)
                frame = cv2.resize(frame, (1280, 720))
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                if frame is None:
                    continue
                video.append(frame)
            else:
                vc.release()
                break
        video_array.append(video)
    return video_array

training_videos_sample = os.listdir(train_videoes)
sample_file_names = training_videos_sample.copy()
complete_paths = []
for i in range(len(training_videos_sample)):
    if training_videos_sample[i].endswith('mp4'):
        complete_paths.append(train_videoes+"/"+training_videos_sample[i])
video_array = get_frames_v1(paths=complete_paths, frames_each_video=20, video_amount=10)

In [ ]:
print(f"Amount of Videos: {len(video_array)}")
print(f"Frames for 6 video: {len(video_array[6])}")

In [ ]:
def plot_video(video: list, figsize: tuple, width: int, height: int) -> None:
    fig = plt.figure(figsize=figsize)
    for i in range(len(video[:(width*height)])):
        plt.subplot(width, height, i+1)
        plt.imshow(video[i])
    plt.show()

plot_video(video_array[6], (30,5), 2, 5)

In [ ]:
def format_frames(vid_arr):
    flatten_first_video_frame = vid_arr[0][0]
    flatten_first_video_frame = flatten_first_video_frame / 255
    return np.array([flatten_first_video_frame])
first_frame_formatted = format_frames(video_array)
first_frame_formatted.shape

### BUILD MODEL
- Add Layers
- Add Loss function, optimizers, and metrics
- Compile model and Fit

In [65]:
model = keras.models.Sequential()
model.add(layers.Conv2D(filters=7, kernel_size=(3,3), padding="same", strides=2, activation="relu", input_shape=first_frame_formatted.shape[1:]))
model.add(layers.MaxPool2D(pool_size=(3,3)))
model.add(layers.Conv2D(filters=3, kernel_size=(3,3), padding="same", activation="relu"))
model.add(layers.MaxPool2D(pool_size=(3,3)))
model.add(layers.Conv2D(filters=3, kernel_size=(3,3), padding="same", activation="relu"))
model.add(layers.MaxPool2D(pool_size=(3,3)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation="relu"))
model.add(layers.Dense(1))
print(model.summary())


Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_46 (Conv2D)          (None, 360, 640, 7)       196       
                                                                 
 max_pooling2d_45 (MaxPoolin  (None, 120, 213, 7)      0         
 g2D)                                                            
                                                                 
 conv2d_47 (Conv2D)          (None, 120, 213, 3)       192       
                                                                 
 max_pooling2d_46 (MaxPoolin  (None, 40, 71, 3)        0         
 g2D)                                                            
                                                                 
 conv2d_48 (Conv2D)          (None, 40, 71, 3)         84        
                                                                 
 max_pooling2d_47 (MaxPoolin  (None, 13, 23, 3)      

In [66]:
loss = keras.losses.BinaryCrossentropy(from_logits=True)
optimizer = keras.optimizers.Adam(learning_rate=0.001)
metrics=["accuracy"]

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)


In [69]:
batch_size = 32
epochs = 5

X = first_frame_formatted
Y =  np.array([1])
print(Y.shape, X.shape)
res = model.fit(X, Y, verbose=2)
print(res.history["accuracy"])

(1,) (1, 720, 1280, 3)
1/1 - 0s - loss: 0.4147 - accuracy: 1.0000 - 80ms/epoch - 80ms/step
[1.0]
